In [2]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import pyarrow.parquet as pq
from pyspark.shell import spark
# Convert parquet to CSV
# df_clicks = spark.read.parquet("data/clicks.parquet")
# df_clicks.write.csv("data_csv/clicks.csv")
#
# df_stores = spark.read.parquet("data/stores.parquet")
# df_stores.write.csv("data_csv/stores.csv")
#
# df_users = spark.read.parquet("data/users.parquet")
# df_users.write.csv("data_csv/users.csv")

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 3.0.1
      /_/

Using Python version 3.8.3 (default, Jul  2 2020 11:26:31)
SparkSession available as 'spark'.


In [3]:
# Visualize Parquet Data in Python
data_clicks = pq.read_table('data/clicks.parquet')
data_clicks_pd = data_clicks.to_pandas()
print("------------data_clicks_pd------------")
print(data_clicks_pd.head())

data_stores = pq.read_table('data/stores.parquet')
data_stores_pd = data_stores.to_pandas()
print("------------data_stores_pd------------")
print(data_stores_pd.head())

data_users = pq.read_table('data/users.parquet')
data_users_pd = data_users.to_pandas()
print("------------data_users_pd------------")
print(data_users_pd.head())

------------data_clicks_pd------------
     index       id  user_id  store_id          device           platform  \
0  1242765   732810    28349       366         app_ios            iOS App   
1   400439  1367874     3553       424  mobile_android                Web   
2   224227  1401055    53657       165         desktop                Web   
3  1628751   867093   236031        36       extension  Browser Extension   
4    34010   568663     3021        36     app_android        Android App   

            channel           created_at  
0            Direct  2021-02-03 23:47:27  
1           Display  2021-03-07 15:52:05  
2          Referral  2021-03-09 13:17:56  
3         Extension  2021-02-09 12:57:00  
4  paid_ins_organic  2021-01-27 00:05:40  
------------data_stores_pd------------
   id  merchant_id             start_at               end_at display_text  \
0   0            0  2020-07-31 15:00:00  2029-12-31 14:59:00     1.0% 캐시백   
1   1            1  2020-03-26 15:00:00  2029-1

In [4]:
#Rename , merge three dataframe , calculate click number of stores
#data_clicks_pd = data_clicks_pd.rename(columns={'user_id': 'userid'})
#data_users_pd = data_users_pd.rename(columns={'id': 'user_id'})
data_stores_pd = data_stores_pd.rename(columns={'id': 'store_id'})
df_merge_clicks_stores = pd.merge(data_clicks_pd, data_stores_pd, on=['store_id', 'store_id'])
df_merge_clicks_stores = df_merge_clicks_stores.rename(columns={'id': 'click_id'})
df_merge_clicks_stores = df_merge_clicks_stores.rename(columns={'user_id': 'id'})
df_merge_clicks_stores_users = pd.merge(df_merge_clicks_stores, data_users_pd, on=['id', 'id'])
df_merge_clicks_stores_users = df_merge_clicks_stores_users.rename(columns={'id': 'user_id'})

dups_store_id = data_clicks_pd.pivot_table(index = ['store_id'], aggfunc='size')
dups_store_id_pd = dups_store_id.to_frame()
df_merge_clicks_stores_users = pd.merge(left= df_merge_clicks_stores_users, right= dups_store_id_pd, how='left', left_on='store_id', right_on='store_id')
df_merge_clicks_stores_users = df_merge_clicks_stores_users.rename(columns={ 0 : 'num_click_store'})

# get a series of merchant_id
merchant_id_list = df_merge_clicks_stores_users.loc[: , "merchant_id"]
merchant_id_list = merchant_id_list.drop_duplicates()
merchant_id_list = merchant_id_list.sort_values()
# for col in df_merge_clicks_stores_users:
#     print(col)

#dups_store_id_pd.columns = ['store_id','total_number']
#index = dups_store_id.Value.argmax()
#print("dups_store_id : ", dups_store_id)
#print(data_clicks_pd.loc[data_clicks_pd['store_id'] == 182 ])
#print('dups_store_id_pd', dups_store_id_pd)
#print('data_clicks_pd.shape : ', data_clicks_pd.shape)
#print('merged_left.shape : ', merged_left.shape)
print(df_merge_clicks_stores_users)
print("merchant_id_list : ", merchant_id_list)

           index  click_id  user_id  store_id       device     platform  \
0        1242765    732810    28349       366      app_ios      iOS App   
1        1242766    838715    28349       366      app_ios      iOS App   
2        1242767   1241814    28349       366      app_ios      iOS App   
3        1242759   1244271    28349       182      app_ios      iOS App   
4        1242758    768524    28349       182      app_ios      iOS App   
...          ...       ...      ...       ...          ...          ...   
1846150  1845088   1053468    21871       281      app_ios      iOS App   
1846151  1845115    471950     3924        14  app_android  Android App   
1846152  1842211   1059179    96410       410      app_ios      iOS App   
1846153  1846110    696918   196455        93      app_ios      iOS App   
1846154  1845114    149047   191857       471   mobile_ios          Web   

                    channel           created_at  merchant_id  \
0                    Direct  2021-

In [ ]:
# import matplotlib as mpl
# mpl.rcParams['agg.path.chunksize'] = 10000
# df_merge_clicks_stores_users.plot.bar(x="store_id", y="num_click_store")

In [287]:
# Task1 : The most clicked merchant for month January 2021
store_id_max = df_merge_clicks_stores_users.loc[df_merge_clicks_stores_users['num_click_store'].idxmax()]
print('store_id_max : ', store_id_max['store_id'])

store_id_max :  182


In [258]:
# Task2 : The distribution of clicks of merchant for month March 2021
# clicked_merchant = df["num_click_store"] > 5000
# store_id_c = df['store_id']
# clicked_merchant.plot( x="store_id_c", y=" clicked_merchant ", kind="bar", rot=5, fontsize=4)

In [259]:
# clicked_merchant = df["num_click_store"] > 5000
# store_id_c = df['store_id']
# clicked_merchant.plot(kind="hist ", rot=5, fontsize=4)

In [288]:
import seaborn as sns
# Task3 :On average, what are the number of clicks user make in 1 month?
Date_df_1 = df_merge_clicks_stores_users[df_merge_clicks_stores_users['created_at'].str.contains("2021-01")]
print('Date_df.shape : ', Date_df_1.shape[0])

Date_df_2 = df_merge_clicks_stores_users[df_merge_clicks_stores_users['created_at'].str.contains("2021-02")]
print('Date_df.shape : ', Date_df_2.shape[0])

Date_df_3 = df_merge_clicks_stores_users[df_merge_clicks_stores_users['created_at'].str.contains("2021-03")]
print('Date_df.shape : ', Date_df_3.shape[0])

ValueError: Could not interpret input 'click_label'

In [261]:
average = ( Date_df_1.shape[0] +Date_df_1.shape[0] +Date_df_1.shape[0] ) / 3
print('The average number of clicks user make in 1 month : ' , average)

The average number of clicks user make in 1 month :  658310.0


In [262]:
# series of all classes of each features
# get a series of merchant_id
print("---------------------device list---------------------")
device_list = df_merge_clicks_stores_users.loc[: , "device"]
device_list = device_list.drop_duplicates()
device_list = device_list.sort_values()
device_list = device_list.to_frame()
device_list['device_label_class'] = range(1, len(device_list) + 1)
print(device_list)

print("----------------------platform list------------------")
platform_list = df_merge_clicks_stores_users.loc[:,"platform"]
platform_list = platform_list.drop_duplicates()
platform_list = platform_list.sort_values()
platform_list = platform_list.to_frame()
platform_list['platform_label_class'] = range(1, len(platform_list)+1)
print(platform_list)

print("----------------------channel list-------------------")
channel_list = df_merge_clicks_stores_users.loc[:,"channel"]
channel_list = channel_list.drop_duplicates()
channel_list = channel_list.sort_values()
channel_list = channel_list.to_frame()
channel_list['channel_label_class'] = range(1, len(channel_list) + 1 )
print(channel_list)

print("----------------------created_at----------------------------")
created_at_list = df_merge_clicks_stores_users.loc[:,"created_at"]
created_at_list = created_at_list.drop_duplicates()
created_at_list = created_at_list.sort_values()
created_at_list = created_at_list.to_frame()
created_at_list = pd.to_datetime(created_at_list['created_at']).astype(int)/ 10**9
#df_merge_clicks_stores_users = pd.to_datetime(df_merge_clicks_stores_users['created_at']).astype(int)/ 10**9
print(created_at_list)

print("----------------------signup_datetime-----------------------")
# signup_date_list = df_merge_clicks_stores_users.loc[:,"signup_datetime"]
# signup_date_list = signup_date_list.drop_duplicates()
# signup_date_list = signup_date_list.sort_values()
# signup_date_list = signup_date_list.to_frame()
#signup_date_list['class_label'] = range(1,len(signup_date_list)+1)
#signup_date_list = pd.to_datetime(signup_date_list['signup_datetime']).astype(int)/ 10**9
#df_merge_clicks_stores_users = pd.to_datetime(df_merge_clicks_stores_users['signup_datetime']).astype(int)/ 10**9
#print(signup_date_list)

print("----------------------lifetime_first_merchant----------------")

# lifetime_first_list = df_merge_clicks_stores_users.loc[:,"lifetime_first_merchant_id"]
# lifetime_first_list = lifetime_first_list.drop_duplicates()
# lifetime_first_list = lifetime_first_list.sort_values()
#lifetime_first_list = lifetime_first_list.to_frame()
#lifetime_first_list['class_label'] = range(1,len(lifetime_first_list)+1)
#lifetime_first_list = pd.to_datetime(lifetime_first_list['lifetime_first_merchant_id']).astype(int)/ 10**9
#df_merge_clicks_stores_users = pd.to_datetime(df_merge_clicks_stores_users['lifetime_first_merchant_id']).astype(int)/ 10**9
#print(lifetime_first_list)

print("----------------------lifetime_first_purchase_datetime-------")
# lifetime_purchase = df_merge_clicks_stores_users.loc[:, "lifetime_first_purchase_datetime"]
# lifetime_purchase = lifetime_purchase.drop_duplicates()
# lifetime_purchase = lifetime_purchase.sort_values()
#lifetime_purchase = lifetime_purchase.to_frame()
#lifetime_purchase['class_label'] = range(1,len(lifetime_purchase)+1)
#lifetime_purchase = pd.to_datetime(lifetime_purchase['lifetime_first_purchase_datetime']).astype(int)/ 10**9
#df_merge_clicks_stores_users = pd.to_datetime(df_merge_clicks_stores_users['lifetime_first_purchase_datetime']).astype(int)/ 10**9
# print(lifetime_purchase)

print("----------------------Account_referral-----------------------")
# account_ref_list = df_merge_clicks_stores_users.loc[:, "account_referral"]
# account_ref_list = account_ref_list.drop_duplicates()
# account_ref_list = account_ref_list.sort_values()
# #account_ref_list = account_ref_list['account_referral'].astype(str).astype(int)
# print(account_ref_list)
#print("--------df_merge_clicks_stores_users--------",df_merge_clicks_stores_users.shape)

---------------------device list---------------------
                 device  device_label_class
156         app_android                   1
0               app_ios                   2
46              desktop                   3
39            extension                   4
125979           mobile                   5
238      mobile_android                   6
219          mobile_ios                   7
1086065          tablet                   8
90212              None                   9
----------------------platform list------------------
              platform  platform_label_class
156        Android App                     1
39   Browser Extension                     2
46                 Web                     3
0              iOS App                     4
----------------------channel list-------------------
                              channel  channel_label_class
4281                          (Other)                    1
0                              Direct                  

# Model Building
0. Dataset Preparation
1. Supervised learning
2. Build a prediction model to predict what is the next merchant a user will click.
3. Serve the model in a web service.

In [263]:
df_merge_clicks_stores_users = pd.merge(left= df_merge_clicks_stores_users, right=device_list, how='left', left_on='device', right_on='device')
df_merge_clicks_stores_users = pd.merge(left= df_merge_clicks_stores_users, right=platform_list,how='left',left_on='platform',right_on='platform' )
df_merge_clicks_stores_users = pd.merge(left= df_merge_clicks_stores_users, right=channel_list,how='left',left_on='channel',right_on='channel' )


#df_merge_clicks_stores_users  = df_merge_clicks_stores_users.sort_values( by=['num_click_store'], inplace=True,ascending=  False )
#new_dataset = new_dataset.drop_duplicates(subset = "store_id")
#print('new_dataset.head(): ', new_dataset.head() )
print('---------------------df_merge_clicks_stores_users-------------')
print(df_merge_clicks_stores_users)
print('df_merge_clicks_stores_users.shape:', df_merge_clicks_stores_users.shape)

---------------------df_merge_clicks_stores_users-------------
           index  click_id  user_id  store_id       device     platform  \
0        1242765    732810    28349       366      app_ios      iOS App   
1        1242766    838715    28349       366      app_ios      iOS App   
2        1242767   1241814    28349       366      app_ios      iOS App   
3        1242759   1244271    28349       182      app_ios      iOS App   
4        1242758    768524    28349       182      app_ios      iOS App   
...          ...       ...      ...       ...          ...          ...   
1846150  1845088   1053468    21871       281      app_ios      iOS App   
1846151  1845115    471950     3924        14  app_android  Android App   
1846152  1842211   1059179    96410       410      app_ios      iOS App   
1846153  1846110    696918   196455        93      app_ios      iOS App   
1846154  1845114    149047   191857       471   mobile_ios          Web   

                    channel         

In [273]:
# Create a new dataset for training and testing
# features select : device, platform, channel
# df_merge_clicks_stores_users = pd.to_datetime(df_merge_clicks_stores_users['created_at']).astype(int)/ 10**9
dataset_features_select = df_merge_clicks_stores_users[[ "click_id", "user_id", "store_id", "device_label_class", "platform_label_class" , "channel_label_class" ,"num_click_store"]]
print(dataset_features_select.head())
#print(dataset_features_select.shape)

   click_id  user_id  store_id  device_label_class  platform_label_class  \
0    732810    28349       366                   2                     4   
1    838715    28349       366                   2                     4   
2   1241814    28349       366                   2                     4   
3   1244271    28349       182                   2                     4   
4    768524    28349       182                   2                     4   

   channel_label_class  num_click_store  
0                    2             2831  
1                   46             2831  
2                   46             2831  
3                   46           259871  
4                    2           259871  


In [276]:
# create a list of our conditions : labels
conditions = [
    (dataset_features_select['num_click_store'] <= 100000),
    (dataset_features_select['num_click_store'] > 100000)
    ]
# create a list of the values we want to assign for each condition
values = ['0', '1']
# create a new column and use np.select to assign values to it using our lists as arguments
dataset_features_select['click_label'] = np.select(conditions, values)
# display updated DataFrame
print(dataset_features_select.head(5))

   click_id  user_id  store_id  device_label_class  platform_label_class  \
0    732810    28349       366                   2                     4   
1    838715    28349       366                   2                     4   
2   1241814    28349       366                   2                     4   
3   1244271    28349       182                   2                     4   
4    768524    28349       182                   2                     4   

   channel_label_class  num_click_store click_label  
0                    2             2831           0  
1                   46             2831           0  
2                   46             2831           0  
3                   46           259871           1  
4                    2           259871           1  


<ipython-input-276-84d4c981a3b4>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_features_select['click_label'] = np.select(conditions, values)


In [277]:
dataset_features_select.to_csv('././data_select_features/new_dataset_features2.csv' , index = False)

In [39]:
# Data Preparation
# TODO1 Combine clicks.parquet and users.parquet table

# Multi class prediction
# TODO1 List all different merchants ID
# TODO2 Select model input features (User ID)
# TODO3 Build Multi class regression model



